In [34]:
from dataclasses import dataclass
@dataclass
class Schema:
    datetime_beginning_utc:str="datetime64[ns]"
    datetime_beginning_ept:str="datetime64[ns]"
    fuel_type:str="string"
    mw:float="float64"
    fuel_percentage_of_total:float="float64"
    is_renewable:bool="boolean"

In [40]:
import pandas as pd
from usagrid import s3


#Generation
fn = s3.list_files_in_folder("usagrid",folder_name="bronze/PJM/generation")[0]
df = s3.read_pyarrow_from_s3(bucket_name="usagrid",key=fn).to_pandas()

In [42]:
schema = Schema().__dict__
for k,v in schema.items():
    
    print(k,v)

    df[k] = df[k].astype(v)

datetime_beginning_utc datetime64[ns]
datetime_beginning_ept datetime64[ns]
fuel_type string
mw float64
fuel_percentage_of_total float64
is_renewable boolean


In [46]:
df_oriented = df.sort_values("datetime_beginning_utc")\
                .reset_index(drop=True).set_index("datetime_beginning_utc")

df_oriented["2023":]

,datetime_beginning_ept,fuel_type,mw,fuel_percentage_of_total,is_renewable
datetime_beginning_utc,,,,,
2023-01-01 00:00:00,2022-12-31 19:00:00,Coal,12857.0,0.140,False
2023-01-01 00:00:00,2022-12-31 19:00:00,Gas,35204.0,0.390,False
2023-01-01 00:00:00,2022-12-31 19:00:00,Hydro,2477.0,0.030,True
2023-01-01 00:00:00,2022-12-31 19:00:00,Multiple Fuels,113.0,0.000,False
2023-01-01 00:00:00,2022-12-31 19:00:00,Nuclear,33947.0,0.380,False
...,...,...,...,...,...
2024-06-19 21:00:00,2024-06-19 17:00:00,Oil,2036.0,0.010,False
2024-06-19 21:00:00,2024-06-19 17:00:00,Other Renewables,626.0,0.000,True
2024-06-19 21:00:00,2024-06-19 17:00:00,Solar,5720.0,0.039,True


In [48]:
s3.write_pyarrow_to_s3(bucket_name="usagrid",object_key="silver/PJM/PJM_generation.arrow",data=df_oriented)